# MVCC (Not in Final)

TODO

# Timestamp Ordering Protocol

- **it is possible to achieve serializability without using locks**


- instead of using locks to prevent conflicting accesses to data, transactions use timestamps to detect such conflicts as they are about to occur; the transaction that discovers the conflict resolves it by aborting itself => can lead to **starvation**


- each transaction is issued a timestamp when it enters the system; if an old transaction $T_i$ has timestamp $TS(T_i)$, a newer transaction $T_j$ is assigned a timestamp $TS(T_j)> TS(T_i)$
- two timestamps are recorded for each data item $Q$
    - **W-timestamp(Q)**: the largest timestamp of any transaction that executed **write(Q)** successfully
    - **R-timestamp(Q)**: the largest timestamp of any transaction that executed **read(Q)** successfully
    - "successfully" means a transaction issues a read/write and completes it without detecting a conflict and aborting
- the timestamp ordering protocol ensures that any conflicting **read** and **write** operations are executed in timestamp order (or not at all)





## Timestamp Ordering Protocol - Read

- suppose a transaction $T_i$ issues a **read(Q)**
    - if TS($T_i$) < W-timestamp(Q), then $T_i$ is attempting to read a value of $Q$ that has already been overwritten
        - in this case the **read** operation is *rejected* and $T_i$ is *rolled back*
    - otherwise $T_i$ is attempting to read the latest value of $Q$, this read operation returns the value of $Q$ and R-timestamp(Q) is set to **max**(R-timestamp(Q), TS($T_i$))
        - in this case the **read** operation is *executed successfully*


$T_i$ | TS($T_i$) < W-timestamp(Q) | TS($T_i)$ $\ge$  W-timestamp(Q)
---|---|---
Result| read is rejected| read successfully, R-timestamp(Q):=**max**(R-timestamp(Q), TS($T_i$))




## Timestamp Ordering Protocol - Write

- suppose a transaction $T_i$ issues **write(Q)**
    - if TS($T_i$) < R-timestamp(Q), then the value of Q that $T_i$ is producing was needed previously, and the system assumed that the value would never be produced
        - the write operation is rejected and $T_i$ is rolled back
    - if TS($T_i$) < W-timestamp(Q), then $T_i$ is attempting to write an obsolete value of Q
        - the write operation is rejected and $T_i$ is rolled back
    - otherwise, the write operation updates Q and W-timestamp(Q) is set to TS($T_i$)
        - executed successfully

$T_i$ | TS($T_i$) < R-timestamp(Q) | TS($T_i$) < W-timestamp(Q)| TS($T_i$)$\ge$ R-timestamp(Q) && TS($T_i$)$\ge$W-timestamp(Q)
---|---|---|---
Result|write is rejected|write is rejected | write successfully, W-timestamp(Q):=TS($T_i$)

## Timestamp Ordering Protocol Correctness

- the timestamp ordering protocol guarantees the edges in the precedence graph are of the form
\begin{equation}
\textrm{transaction with smaller timstamp} \rightarrow \textrm{transaction with larger timestamp}
\end{equation}

- since edges always point from older transactions to newer ones, there can be no cycles in the precedence graph, thus timestamp ordering **guarantees conflict serializability**

- timestamp ordering also ensures **freedom from deadlock** since no transaction ever waits for another (e.g., to release a lock)

- but the schedule **may not be cascadeless**, and **may not even be recoverable**, also **starvation** may occur if the same transaction (e.g., large read-only transaction) aborts repeatedly

### Recoverbility Problem

- suppose $T_i$ aborts, but $T_j$ has read a data item written by $T_i$, then $T_j$ must abort; if $T_j$ had been allowed to commit earlier, the schedule is not recoverable

**Solution**: keep track of commit dependencies and delay commitment of a transaction until all of the transactions it depends on have committed => ensures recoverable schedules

### Cascade-Freedom Problem

- suppose $T_i$ aborts, but $T_j$ has read a data item written by $T_i$, then $T_j$ must abort; furthermore, any transaction that has read a data item written by $T_j$ must abort, which can lead to cascading rollback

**Solution**: A transaction is structured such that all of its writes are performed at the end, atomically with the commit step => ensure cascadeless schedules

## Thomas' Write Rule

- modified version of the timestamp ordering protocol in which obsolete **write** operations may be ignored under certain circumstances


- **Thomas' Write Rule**: when $T_i$ attempts to write data item $Q$, if $\textrm{TS}(T_i) < \textrm{W-timestamp}(Q)$, then $T_i$ is attempting to write an obsolete value of $Q$; rather than rolling back $T_i$, we can simply ignore the **write** operation (i.e., treat it as a no-op) provided that $\textrm{TS}(T_i) \ge \textrm{R-timestamp}(Q)$
    - enables slightly greater concurrency, and permits some view-serializable schedules that are not conflict serializable, e.g.:
    
<img src="img/Snip20191129_91.png" width=50%/>



<img src="img/Snip20191129_92.png" width=80%/>
<img src="img/Snip20191129_93.png" width=80%/>

# Deadlock Handling

<img src="img/Snip20191129_94.png" width=60%/>

- system is deadlocked if there is a set of transactions such that every transaction in the set is waiting for another transaction in the set


- **Deadlock prevention** protocols ensure that the system will never enter into a deadlock state
    - **pre-declaration**: require that each transaction locks all its data items before it begins execution
    - **lock ordering**: impose a partial order on all data items and require that a transaction lock data items in the order specified
    - **timeout-based**: a transaction waits for a lock only for a specified amount of time, then rolls back if lock is not granted
- **Deadlock detection** protocols allow deadlock to occur, detect when it has occurred, and then recover by rolling back some transaction


## Remark

- deadlock avoidance is preferale if the consequences of abort are serious, and if there is high contention and a resulting high probability of deadlock



## Wait-die (non-preemptive) & Wound-wait (preemptive)

- deadlock prevention strategies that incorporate transaction timestamps that indicate the time when a transaction became active
- both wait-die and wound-wait **avoid starvation**



- **Wait-die**: older transaction may wait for younger one to release a lock; younger transactions never wait for older ones, they are rolled back (i.e., die) instead
    - a transaction may die several times before acquiring a needed lock



- **Wound-wait**: older transaction *wounds* (forces rollback of) younger transaction instead of waiting for it; younger transactions may wait for older ones
    - potentially fewer rollbacks than *wait-die* scheme
    - used by Google Spanner




# Deadlock Detection


- deadlocks can be detected by constructing a **wait-for graph** $G(V, E)$ where
    - $V$ is a set of vertices representing the transactions in the system
    - $E$ is a set of directed edges representing wait-for dependencies where $T_i\rightarrow T_j$ is in $E$ if $T_i$ is waiting for $T_j$ to release a data item


- when $T_i$ requests a data item currently being held by $T_j$, then the edge $T_i\rightarrow T_j$ is inserted into the wait-for graph; this edge is removed only when $T_i$ stops waiting, such as when $T_j$ is no longer holding a lock on a data item needed by $T_i$


- the system is in a **deadlock state if and only if the wait-for graph has a cycle**
    - the DBMS must invoke a deadlock detection algorithm periodically to check for cycles
    
<img src="img/Snip20191129_96.png" width=80%/>

# Deadlock Recovery

- some transaction must be rolled back (i.e., made a victim) to break the deadlock
- the victim is chosen to be the transaction whose rollback will incur minimum cost
- starvation happens if the same transaction is chosen as victim repeatedly; the number of rollbacks can be included in the cost calculation to avoid starvation
- the victim may be rolled back either partially or completely
    - total rollback: abort the transaction and then restart it
    - partial rollback: roll back the transaction only as far as necessary to break the deadlock